In [1]:
import dask.array as da
import dask
import zarr
import cv2
import numpy as np
import matplotlib

In [3]:
from MotionEnergyAnalyzer import MotionEnergyAnalyzer
import utils

In [4]:
# notebook dev
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:

zarr_paths = utils.find_zarr_paths()

Searching for Zarr directories in /root/capsule/data: 100%|██████████| 1/1 [00:00<00:00, 20460.02it/s]
Searching for Zarr directories in /root/capsule/data/test_data_1: 100%|██████████| 1/1 [00:00<00:00, 11066.77it/s]
Searching for Zarr directories in /root/capsule/data/test_data_1/processed_frames_zarr: 100%|██████████| 1/1 [00:00<00:00, 20068.44it/s]
Searching for Zarr directories in /root/capsule/data/test_data_1/processed_frames_zarr/data: 0it [00:00, ?it/s]

Found Zarr directory: /root/capsule/data/test_data_1/processed_frames_zarr


In [6]:
analyser = MotionEnergyAnalyzer(zarr_paths[0])

In [7]:
analyser.analyze()

<class 'dask.array.core.Array'>
<class 'dask.array.core.Array'>


KeyboardInterrupt: 

In [48]:
 grayscale_frames = da.from_zarr(analyser.zarr_store_frames, component='data')

In [ ]:

# Load the grayscale frames from the Zarr file
zarr_store_frames = zarr.DirectoryStore('video_frames_grayscale.zarr')
grayscale_frames = da.from_zarr(zarr_store_frames, component='frames')

# Compute motion energy as the absolute difference between consecutive frames
# This will be a Dask array operation that only loads and computes chunks as needed
motion_energy = da.abs(grayscale_frames[1:] - grayscale_frames[:-1])

# Define the chunking for efficient memory usage; rechunk if necessary
motion_energy = motion_energy.rechunk((100, 658, 492))  # Adjust based on available memory

# Save motion energy as a new Zarr file
zarr_store_motion = zarr.DirectoryStore('motion_energy_grayscale.zarr')
motion_energy.to_zarr(zarr_store_motion, overwrite=True)

# Compute the sum of motion energy for each frame and save as a NumPy array
# This sums across height and width (658, 492) to get the sum per frame (along the time dimension)
motion_energy_sum = motion_energy.sum(axis=(1, 2)).compute()

# Save the result as a NumPy array
np.save('motion_energy_sum.npy', motion_energy_sum)